# Assignment10
## Name : Yeon-Jee Jung
## Student ID : 20142052
## Git URL : https://github.com/YeonjeeJung/assignment10

# Import packages for plotting graphs and manipulating data :

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.linalg as spla

# Setting for input data :
## Open input files

In [2]:
file_data_train = "mnist_train.csv"
file_data_test = "mnist_test.csv"

h_data_train = open(file_data_train, 'r')
h_data_test = open(file_data_test, 'r')

data_train = h_data_train.readlines()
data_test = h_data_test.readlines()

h_data_train.close()
h_data_test.close()

size_row = 28 # height of the image
size_col = 28 # width of the image

num_train = len(data_train) # number of training images
num_test = len(data_test) # number of testing images

## Define normalization function
Make input data to be in $[0, 1]$

In [3]:
#
# normalize the values of the input data to be [0, 1]
#

def normalize(data):
    data_normalized = (data  - min(data)) / (max(data) - min(data))
    return data_normalized

## Make train and test data matrix with input file
In MNIST, 1 image size is $28\times28$. In matrix, one column represents one image. So train data matrix size is $784\times num\_train$, and test data matrix size is $784\times num\_test$.

$matrix = \begin{bmatrix}x_1 & x_2 & \cdots & x_{num} \end{bmatrix}$, where $x_k$ is vector of one image.

If the label is '0', then final label is '1'(true). If the label is not '0', then final label is '-1'(false). 

In [4]:
def makeTrainTestSet(i, data_train, data_test):
    #
    # make a matrix each column of which represents an images in a vector form
    #

    list_image_train = np.empty((size_row * size_col, num_train), dtype=float)
    list_label_train = np.empty(num_train, dtype=int)

    list_image_test = np.empty((size_row * size_col, num_test), dtype=float)
    list_label_test = np.empty(num_test, dtype=int)

    count = 0

    for line in data_train:
        line_data = line.split(',')
        label = line_data[0]

        if label == str(i):
            label = 1
        else:
            label = -1

        im_vector = np.asfarray(line_data[1:])
        im_vector = normalize(im_vector)

        list_label_train[count] = label
        list_image_train[:, count] = im_vector

        count += 1

    count = 0

    for line in data_test:
        line_data = line.split(',')
        label = line_data[0]

        if label == str(i):
            label = 1
        else:
            label = -1

        im_vector = np.asfarray(line_data[1:])
        im_vector = normalize(im_vector)

        list_label_test[count] = label
        list_image_test[:, count] = im_vector

        count += 1
        
    return list_image_train, list_label_train, list_image_test, list_label_test

# Define functions to solve Least_square problem :
We will find 
$\begin{bmatrix} s_1 \\ s_2 \\ \vdots \\ s_p \end{bmatrix}$ that minimizes $\begin{Vmatrix}\begin{bmatrix}f_1(x_1) & f_2(x_1) &\cdots &f_p(x_1) \\f_1(x_2) & f_2(x_2) &\cdots &f_p(x_2) \\\vdots & \vdots &&\vdots \\f_1(x_n) & f_2(x_n) &\cdots&f_p(x_n)\end{bmatrix}\cdot\begin{bmatrix} s_1 \\ s_2 \\ \vdots \\ s_p\end{bmatrix} - \begin{bmatrix} y_1 \\ y_2 \\ \vdots \\ y_n \end{bmatrix}\end{Vmatrix}$, 
Where $f_k(x)=\begin{bmatrix} r_1 \\ r_2 \\ \vdots \\ r_{784} \end{bmatrix} \cdot \begin{bmatrix}x_1 \\ x_2 \\ \vdots \\ x_{784}\end{bmatrix}$, $r_i$ is random normal number.

## Define norm function
This function returns norm of a vector.

In [5]:
def norm(vector):
    sum = 0
    for i in range(len(vector)):
        sum += vector[i]**2
    return np.sqrt(sum)

## Define proj function
This function returns projection of vectors.

In [6]:
def proj(e, a):
    return (np.matmul(e.T, a) / np.matmul(e.T, e))*e

## Defind randomExtractor function
This function makes random feature-extraction vectors.

In [7]:
def randomExtractor(p):
    extractor = np.random.normal(0, 1, (p, 784))
    return extractor

## Define findX function
This function returns x, from Q, R, and b. Q, R is from computeQR function, that makes A by multiplication. So this function can find x that satisfy least-square problem such that $A\cdot x = Q\cdot R\cdot x = b$.

In [8]:
def findX(Q, R, b):
    Rsol = np.matmul(Q.T, b)
    sol = np.zeros(Rsol.shape)
    for i in reversed(range(Rsol.shape[0])):
        a = Rsol[i]
        for j in reversed(range(i+1, Rsol.shape[0])):
            a -= sol[j]*R[i][j]
        if R[i][i] == 0:
            sol[i] = 0
        else:
            sol[i] = a / R[i][i]
    return sol

## Define F1score function
This function calculates F1 score. $F_1 = 2 \cdot \frac{precision\cdot recall}{precision+recall}$ where $precision = \frac{TP}{TP+FP}$, $recall = \frac{TP}{TP+FN}$

In [9]:
def F1score(list_TFcount):
    precision = list_TFcount[0] / (list_TFcount[0] + list_TFcount[3])
    recall = list_TFcount[0] / (list_TFcount[0] + list_TFcount[1])
    return 2*((precision * recall) / (precision + recall))

## Define TF function
This function predicts the test dataset, and evaluate the score. TP is for True Positive, TN is for True Negative, FP is for False Positive, FN is for False Negative.

In [10]:
def TF(RE, sol, image_test, label_test):
    Test = np.matmul(RE, image_test)
    pred = np.sign(np.matmul(Test.T, sol))

    list_TF = [None for i in range(num_test)]
    for i in range(num_test):
        if pred[i] == 1.0 and label_test[i] == 1:
            list_TF[i] = 'TP'
        elif pred[i] == -1.0 and label_test[i] == 1:
            list_TF[i] = 'FN'
        elif pred[i] == -1.0 and label_test[i] == -1:
            list_TF[i] = 'TN'
        elif pred[i] == 1.0 and label_test[i] == -1:
            list_TF[i] = 'FP'

    TFcount = [
        list_TF.count('TP'), list_TF.count('FP'), 
        list_TF.count('TN'), list_TF.count('FN')
    ]
    print("(TP, FP, TN, FN) :", TFcount)
    
    F1 = F1score(TFcount)
    print("F1 score :", F1)

    return list_TF, F1

# Solve Least-square problem for classification:
## Calculate F1 score for each i, and predict results for test set.

In [11]:
preds = []

for i in range(10):
    img_tr, lbl_tr, img_te, lbl_te = makeTrainTestSet(i, data_train, data_test)
    
    extractor = randomExtractor(818)
    A = np.matmul(extractor, img_tr)
    Q, R = np.linalg.qr(A.T)
    sol = findX(Q, R, lbl_tr)
    list_TF, F1 = TF(extractor, sol, img_te, lbl_te)
    
    Test = np.matmul(extractor, img_te)
    p = np.matmul(Test.T, sol)
    preds.append(p)
    print("Finished", i)
    

(TP, FP, TN, FN) : [926, 73, 8947, 54]
F1 score : 0.9358261748357756
Finished 0
(TP, FP, TN, FN) : [1054, 76, 8789, 81]
F1 score : 0.9306843267108168
Finished 1
(TP, FP, TN, FN) : [666, 31, 8937, 366]
F1 score : 0.7703875072296125
Finished 2
(TP, FP, TN, FN) : [657, 78, 8912, 353]
F1 score : 0.7530085959885388
Finished 3
(TP, FP, TN, FN) : [744, 71, 8947, 238]
F1 score : 0.8280467445742904
Finished 4
(TP, FP, TN, FN) : [444, 63, 9045, 448]
F1 score : 0.6347390993566834
Finished 5
(TP, FP, TN, FN) : [790, 102, 8940, 168]
F1 score : 0.8540540540540541
Finished 6
(TP, FP, TN, FN) : [778, 63, 8909, 250]
F1 score : 0.8325307651150348
Finished 7
(TP, FP, TN, FN) : [444, 149, 8877, 530]
F1 score : 0.566687938736439
Finished 8
(TP, FP, TN, FN) : [583, 141, 8850, 426]
F1 score : 0.6728216964800924
Finished 9


## Find the argmax index, predict the final result.

In [12]:
preds = np.array(preds)
pred = np.empty(num_test, dtype=int)
for i in range(num_test):
    pred[i] = int(np.argmax(preds[:,i]))
print(pred)

[7 2 1 ... 4 5 6]


## Make the true number label of the test set.

In [13]:
list_real_label_test = np.empty(num_test, dtype=int)

count = 0
for line in data_test:
    line_data = line.split(',')
    label = line_data[0]

    list_real_label_test[count] = label

    count += 1

## Plot the confusion Matrix.

In [14]:
confusionMatrix = np.zeros((10, 10), dtype=int)

for i in range(num_test):
    confusionMatrix[list_real_label_test[i], pred[i]] += 1
    
for i in range(10):
    for j in range(10):
        print("%4d"%confusionMatrix[i, j], end=" ")
    print("")

 944    0    0    2    0    6   13    1   13    1 
   0 1105    2    2    2    1    5    1   16    1 
  19   55  808   28   18    0   40   18   40    6 
   9   21   22  867    3   20    9   22   24   13 
   1   24    5    3  861    6    9    2   16   55 
  18   15    3   85   18  620   19   13   78   23 
  27    9    9    1   21   19  863    0    8    1 
   6   39   14    9   23    0    1  860    5   71 
  21   54   10   26   25   41   14   10  750   23 
  16   11    1   15   68    1    1   63   18  815 
